<a href="https://colab.research.google.com/github/biranchi2018/My-NLP-Examples/blob/master/3.BERT%20-%20Masked_Language_Model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 665kB 6.3MB/s 
     |████████████████████████████████| 1.1MB 30.9MB/s 
     |████████████████████████████████| 890kB 19.8MB/s 
     |████████████████████████████████| 3.8MB 40.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=58c5a57455bfc06d54f524182fb61926c3cb21b1ce6c9c02d98845a08070cd00
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


BERT - Masked Language Model
=============================

https://becominghuman.ai/practical-ai-using-pretrained-bert-to-generate-grammar-and-vocabulary-multiple-choice-questions-d92e4fbeeeaa


In [0]:
import re
import torch
from transformers import BertTokenizer, BertForMaskedLM

In [3]:
import time
start = time.time()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()
end = time.time()
print ("Time Elapsed to load BERT ",end-start)


Time Elapsed to load BERT  15.730551719665527


In [0]:
def get_predicted_words(text):
    text = "[CLS] " + text.replace("____", "[MASK]") + " [SEP]"
    # text= '[CLS] Tom has fully [MASK] from his illness. [SEP]'
    
    tokenized_text = tokenizer.tokenize(text)
    
    #print("tokenized sentence: ",tokenized_text,"\n")
    masked_index = tokenized_text.index('[MASK]')
    print(f'masked_index : {masked_index}')
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

	  # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

	  # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

	  # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)[0]
	
	  # Get 30 choices for the masked(blank) word
    #print(f'predictions : {predictions}') 
    k = 30
    predicted_index, predicted_index_values = torch.topk(predictions[0, masked_index], k)
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_index_values.tolist())
    filtered_tokens_to_remove_punctuation = []
    
    # Remove any predictions that contain punctuation etc as they are not relevant to us.
    for token in predicted_tokens:
        if re.match("^[a-zA-Z0-9_]*$", token):
            filtered_tokens_to_remove_punctuation.append(token)
        
    return filtered_tokens_to_remove_punctuation

In [20]:
sentence = "They all look tiny ____ they are so far away from the Earth."
print("original sentence: ",sentence,"\n")

original sentence:  They all look tiny ____ they are so far away from the Earth. 



In [24]:
predicted_words = get_predicted_words(sentence)
print()
print("predicted choices:\n", predicted_words)

masked_index : 5
predictions : tensor([[[ -6.5230,  -6.5132,  -6.5371,  ...,  -5.8959,  -5.7181,  -3.9671],
         [-13.1075, -13.0090, -12.6393,  ..., -10.3502, -11.1232, -10.2471],
         [-14.8110, -15.1335, -14.6437,  ..., -13.3993, -13.3356, -13.7805],
         ...,
         [-11.3714, -11.3986, -11.2618,  ...,  -8.9173,  -9.7927,  -8.4863],
         [-14.4480, -14.0974, -14.4010,  ..., -12.0537, -11.8662,  -9.4477],
         [-19.7119, -19.8447, -19.6689,  ..., -17.6284, -17.9101, -11.4495]]])

predicted choices:
 ['because', 'and', 'but', 'as', 'since', 'like', 'when', 'considering', 'for', 'except', 'though', 'that', 'yet', 'although', 'while', 'now', 'if', 'where', 'here', 'even', 'so', 'after', 'from', 'given']
